# 📊 Trend vs Review Rating Analysis (GitHub Portfolio Notebook)

**목표:** 주별 트렌드(수요) 지수 변화가 리뷰 평점(경험 품질)에 어떤 관계가 있는지 검증하고, PM 관점의 판단/의사결정까지 연결합니다.

- 데이터: `baemin.csv`(리뷰 130K+) + `bamin_trend_expanded.csv`(주별 트렌드)
- 분석 단위: ISO Year-Week(연-주)
- 핵심 산출물: **그래프 3종 + 상관분석 + PM 인사이트**

## 0) 환경 설정: 한글 폰트(필수)

그래프에서 한글이 깨지면 **포트폴리오 신뢰도가 바로 떨어집니다.**  
아래 셀은 OS에 따라 한글 폰트를 자동 선택합니다.

- Windows: `Malgun Gothic`
- macOS: `AppleGothic`
- Linux/Colab: `NanumGothic` (필요 시 설치)

> ⚠️ Linux에서 `NanumGothic`이 없으면, 아래 주석의 설치 명령을 사용하세요.

In [ ]:
# (Linux/Colab) 필요 시 폰트 설치
# !apt-get update -y
# !apt-get install -y fonts-nanum

import os
import platform
import matplotlib.pyplot as plt
from matplotlib import rcParams, font_manager

def set_korean_font():
    system = platform.system()
    candidates = []
    if system == "Windows":
        candidates = ["Malgun Gothic"]
    elif system == "Darwin":  # macOS
        candidates = ["AppleGothic"]
    else:  # Linux
        candidates = ["NanumGothic", "Noto Sans CJK KR", "NotoSansCJKkr"]

    available = {f.name for f in font_manager.fontManager.ttflist}
    for c in candidates:
        if c in available:
            rcParams["font.family"] = c
            break

    rcParams["axes.unicode_minus"] = False

set_korean_font()

## 1) 데이터 로드

> 아래 경로는 GitHub 리포지토리 구조를 기준으로 작성했습니다.

```
repo/
  data/baemin.csv
  data/bamin_trend_expanded.csv
  outputs/
  notebooks/analysis.ipynb
```

In [ ]:
import pandas as pd
import numpy as np

REVIEWS_PATH = "../data/baemin.csv"
TREND_PATH   = "../data/bamin_trend_expanded.csv"

reviews = pd.read_csv(REVIEWS_PATH, parse_dates=["at"])
trend   = pd.read_csv(TREND_PATH)

print("reviews:", reviews.shape)
print("trend:", trend.shape)

reviews.head()

## 2) 주별(ISO Year-Week) 집계

- **주별 평균 평점(avg_score)**
- **주별 리뷰 수(review_count)**

이 두 값은 PM 관점에서 다음 역할을 합니다.
- `avg_score`: 경험 품질(결과 지표)
- `review_count`: 수요/부하(상황 지표)

In [ ]:
reviews["iso_year"] = reviews["at"].dt.isocalendar().year.astype(int)
reviews["iso_week"] = reviews["at"].dt.isocalendar().week.astype(int)

weekly_reviews = (
    reviews.groupby(["iso_year","iso_week"])
    .agg(avg_score=("score","mean"),
         review_count=("score","size"))
    .reset_index()
)

weekly_reviews.head()

## 3) 트렌드 데이터도 동일 주차로 집계 후 병합

트렌드 데이터에서 **대표 지수 컬럼**은 기본적으로 `전체`를 사용합니다.  
(만약 컬럼명이 다르면 아래 셀에서 자동으로 첫 번째 숫자 컬럼을 사용하도록 되어 있습니다.)

In [ ]:
trend["date"] = pd.to_datetime(trend["date"])
trend["iso_year"] = trend["date"].dt.isocalendar().year.astype(int)
trend["iso_week"] = trend["date"].dt.isocalendar().week.astype(int)

numeric_cols = trend.select_dtypes(include=[np.number]).columns.tolist()
trend_metric_col = "전체" if "전체" in trend.columns else (numeric_cols[0] if numeric_cols else None)
assert trend_metric_col is not None, "트렌드 데이터에서 숫자 컬럼을 찾지 못했습니다."

weekly_trend = (
    trend.groupby(["iso_year","iso_week"])
    .agg(trend_mean=(trend_metric_col,"mean"))
    .reset_index()
)

merged = weekly_reviews.merge(weekly_trend, on=["iso_year","iso_week"], how="inner")
merged["week_index"] = merged["iso_year"]*100 + merged["iso_week"]

merged.head()

## 4) 그래프 1 — 주별 추이(라인)

**이 그래프에서 봐야 할 것**
1. 트렌드 지수가 상승하는 구간에서 평균 평점이 동반 하락하는지  
2. 특정 주차에 “리뷰 수 급증 + 평점 급락”이 동시에 발생하는지  
3. 패턴이 반복되는지(일시적 이슈 vs 구조적 이슈)

In [ ]:
import os
os.makedirs("../outputs", exist_ok=True)

plt.figure(figsize=(12,6))
plt.plot(merged["week_index"], merged["avg_score"], label="주별 평균 평점", linewidth=2)
plt.plot(merged["week_index"], merged["trend_mean"], label=f"주별 트렌드 지수({trend_metric_col})", linewidth=2, alpha=0.75)
plt.title("주별 트렌드 지수와 평균 평점 변화")
plt.xlabel("주차(YYYYWW)")
plt.ylabel("값")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig("../outputs/01_주별_트렌드_평점_추이.png", dpi=200)
plt.show()

## 5) 상관관계 분석 (Pearson / Spearman)

- Pearson: 선형 관계(크기 포함)  
- Spearman: 순위 기반(비선형/이상치에 조금 더 강함)

**해석 가이드**
- r < 0 이면: 트렌드 상승 ↔ 평점 하락(음의 상관)
- p-value가 작으면: 우연일 가능성이 낮음(통계적 유의)

In [ ]:
from scipy.stats import pearsonr, spearmanr

pear_r, pear_p = pearsonr(merged["trend_mean"], merged["avg_score"])
spear_r, spear_p = spearmanr(merged["trend_mean"], merged["avg_score"])

result = pd.DataFrame({
    "metric": ["Pearson r", "Spearman ρ"],
    "value":  [pear_r, spear_r],
    "p_value":[pear_p, spear_p],
})
result

## 6) 그래프 2 — 트렌드 vs 평점(산점도)

**이 그래프에서 봐야 할 것**
1. 점들이 전체적으로 우하향하는지(음의 상관 시각적 확인)  
2. 특정 구간에서 분산이 커지는지(수요 구간별 경험 변동성)  
3. 선형이 아닌 경우(임계점/비선형 패턴) 가능성

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(merged["trend_mean"], merged["avg_score"], alpha=0.5)
plt.title("트렌드 지수와 평균 평점의 관계(산점도)")
plt.xlabel(f"주별 트렌드 지수({trend_metric_col})")
plt.ylabel("주별 평균 평점")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("../outputs/02_트렌드_vs_평점_산점도.png", dpi=200)
plt.show()

## 7) 그래프 3 — 리뷰 수 × 평점 × 트렌드(3변수)

이 그래프는 “**왜** 평점이 떨어지는가”에 가장 가까운 힌트를 줍니다.

- x축: 리뷰 수(수요/부하)  
- y축: 평균 평점(경험 품질)  
- 색상: 트렌드 지수(수요 환경)

**이 그래프에서 봐야 할 것**
1. 리뷰 수가 높고(부하↑), 트렌드가 높을수록(색상↑) 평점이 낮아지는 패턴이 있는지  
2. 평점 방어가 성공한 주차가 존재하는지(비교 대상)  
3. “수요 증가 자체”가 아니라 “수요 감당 실패”가 핵심이라는 근거

In [ ]:
plt.figure(figsize=(10,6))
sc = plt.scatter(merged["review_count"], merged["avg_score"], c=merged["trend_mean"], alpha=0.65)
cb = plt.colorbar(sc)
cb.set_label(f"트렌드 지수({trend_metric_col})")
plt.title("리뷰 수, 평균 평점, 트렌드 지수의 관계")
plt.xlabel("주별 리뷰 수")
plt.ylabel("주별 평균 평점")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("../outputs/03_리뷰수_평점_트렌드_관계.png", dpi=200)
plt.show()

## 8) PM 관점 결론 (Portfolio Narrative)

**결론(한 줄)**  
트렌드(수요) 상승 국면에서 평균 평점이 하락하는 **유의미한 음의 상관관계**가 관찰되었고, 특히 **리뷰 수(부하)가 큰 주차**에서 하락 폭이 커지는 경향이 나타났다.

**판단(Judgment)**  
- 문제의 본질은 기능 부족이 아니라, 수요 급증 상황에서 발생하는 **운영/UX 병목 + 기대 관리 실패** 가능성이 큼  
- 따라서 해결 방향은 “기능 추가”가 아니라 **주문 이전 단계에서의 정보 제공 타이밍(경험 순서) 재설계**

**Next Steps(후속 실험)**  
1) 평점 하락 주차 Top 10 텍스트 토픽/감성 분석 → 원인 가설 정교화  
2) 트렌드 대비 평점 방어 성공/실패 주차 비교(시간대/지역/가맹점 특성)  
3) 주문 전 ETA/혼잡도/대체옵션 노출 A/B 테스트로 평점 방어 효과 검증